In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 57.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random

##Extracting Featurse from Mediapipe

In [ ]:
parent_dir = '/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data'
excel_path = '/content/drive/MyDrive/Anit/Gym_Posture_Data/candidates list.xlsx'

df = pd.read_excel(excel_path)
vid_names = df['VidName'].to_list()

In [ ]:
labels=df['Label'].to_list()
print(labels)

print('zeros',labels.count(0))
print('ones',labels.count(1))


[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
zeros 25
ones 43


In [ ]:
len(vid_names)

68

In [ ]:
os.chdir(f'/content/drive/MyDrive/{}/Gym_Posture_Data/Extracted_Features')
!pwd

FileNotFoundError: ignored

In [ ]:
# Iterate through video names
for vid_name in vid_names:
    test_video = os.path.join(parent_dir, vid_name)

    print(test_video)
    # Initialize MediaPipe Pose module
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Open the video file
    video_capture = cv2.VideoCapture(test_video)

    landmarks_data = []

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        # Convert the frame to RGB format for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect poses in the frame
        results = pose.process(frame_rgb)

        if results.pose_landmarks is not None:
            landmarks = []
            for landmark in results.pose_landmarks.landmark:
                landmarks.append((landmark.x, landmark.y, landmark.z))

            landmarks_data.append(landmarks)

        # Exit the video by pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video_capture.release()
    cv2.destroyAllWindows()


    # Convert landmarks data to a NumPy array
    landmarks_array = np.array(landmarks_data)

    # Construct the file name including the video name
    save_file_name = f'{vid_name}_landmarks.npy'

    # Save the landmarks data to a NumPy file
    np.save(save_file_name, landmarks_array)

/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/alli1.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/alli2.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/alli3.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anan1.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anan2.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anan3.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anit1.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anit2.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anit3.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/ank1.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/ank2.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/ank3.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anu1.mov
/content/drive/MyDrive/Anit/Gym_Posture_Data/gym clean data/anu2.mov
/content/drive/MyDrive/An

##Checking the Files which have zero shape

In [ ]:
folder_path = '/content/drive/MyDrive/Anit/Gym_Posture_Data/Extracted_Features'
list_npy = os.listdir(folder_path)

for idx, file_npy in enumerate(list_npy):
    file_path = os.path.join(folder_path, file_npy)
    file_loaded = np.load(file_path)
    if file_loaded.shape == (0,):
        print(f"File at index {idx} has a shape of (0,): {file_npy}")

File at index 2 has a shape of (0,): alli3.mov_landmarks.npy
File at index 23 has a shape of (0,): chi3.mov_landmarks.npy
File at index 35 has a shape of (0,): mango3.mov_landmarks.npy
File at index 47 has a shape of (0,): peher3.mov_landmarks.npy


##Removing the zero shape file

In [ ]:
filtered_list_npy = [file_npy for file_npy in list_npy if np.load(os.path.join(folder_path, file_npy)).shape != (0,)]
print(len(filtered_list_npy))

values_list = []

for target_value in filtered_list_npy:
    target_value = target_value.split('_')[0]
    values = df.loc[df['VidName'] == target_value, 'Label'].tolist()
    values_list.extend(values)  # Add extracted values to the list

print("Length of values_list:", len(values_list))
print("Length of list_npy:", len(filtered_list_npy))

64
Length of values_list: 64
Length of list_npy: 64


['alli1.mov_landmarks.npy',
 'alli2.mov_landmarks.npy',
 'anan1.mov_landmarks.npy',
 'anan2.mov_landmarks.npy',
 'anan3.mov_landmarks.npy',
 'anit1.mov_landmarks.npy',
 'anit2.mov_landmarks.npy',
 'anit3.mov_landmarks.npy',
 'ank1.mov_landmarks.npy',
 'ank2.mov_landmarks.npy',
 'ank3.mov_landmarks.npy',
 'anu1.mov_landmarks.npy',
 'anu2.mov_landmarks.npy',
 'anu3.mov_landmarks.npy',
 'arvind1.mov_landmarks.npy',
 'arvind2.mov_landmarks.npy',
 'arvind3.mov_landmarks.npy',
 'bhave1.mov_landmarks.npy',
 'bhave2.mov_landmarks.npy',
 'bhave3.mov_landmarks.npy',
 'chi1.mov_landmarks.npy',
 'chi2.mov_landmarks.npy',
 'cos1.mov_landmarks.npy',
 'cos2.mov_landmarks.npy',
 'cos3.mov_landmarks.npy',
 'daga1.mov_landmarks.npy',
 'daga2.mov_landmarks.npy',
 'daga3.mov_landmarks.npy',
 'man1.mov_landmarks.npy',
 'man2.mov_landmarks.npy',
 'man3.mov_landmarks.npy',
 'mango1.mov_landmarks.npy',
 'mango2.mov_landmarks.npy',
 'minku1.mov_landmarks.npy',
 'minku2.mov_landmarks.npy',
 'minku3.mov_landmark

In [ ]:
# Create a new shuffled list
shuffled_list = random.sample(filtered_list_npy, len(filtered_list_npy))

print("Original list:", filtered_list_npy)
print("Shuffled list:", shuffled_list)


Original list: ['y2.mov_landmarks.npy', 'sub2.mov_landmarks.npy', 'sahu3.mov_landmarks.npy', 'daga3.mov_landmarks.npy', 'sahu1.mov_landmarks.npy', 'anan1.mov_landmarks.npy', 'vik2.mov_landmarks.npy', 'nik1.mov_landmarks.npy', 'arvind1.mov_landmarks.npy', 'cos3.mov_landmarks.npy', 'nik3.mov_landmarks.npy', 'vik1.mov_landmarks.npy', 'vik3.mov_landmarks.npy', 'anu2.mov_landmarks.npy', 'anan2.mov_landmarks.npy', 'bhave3.mov_landmarks.npy', 'cos2.mov_landmarks.npy', 'anit3.mov_landmarks.npy', 'shashank1.mov_landmarks.npy', 'minku3.mov_landmarks.npy', 'minku2.mov_landmarks.npy', 'man2.mov_landmarks.npy', 'cos1.mov_landmarks.npy', 'arvind2.mov_landmarks.npy', 'shashank2.mov_landmarks.npy', 'par3.mov_landmarks.npy', 'bhave1.mov_landmarks.npy', 'man1.mov_landmarks.npy', 'tlp1.mov_landmarks.npy', 'sahu2.mov_landmarks.npy', 'chi1.mov_landmarks.npy', 'sub3.mov_landmarks.npy', 'alli1.mov_landmarks.npy', 'sub1.mov_landmarks.npy', 'par1.mov_landmarks.npy', 'ank1.mov_landmarks.npy', 'anan3.mov_landmar

In [ ]:
extracted_feature_path = '/content/drive/MyDrive/Anit/Gym_Posture_Data/Extracted_Features'


avg_arr = []
for idx in range(0,64):

  file_path = os.path.join(extracted_feature_path,filtered_list_npy[idx])
  # print(file_path)

  file = np.load(file_path)
  # print(file.shape[0])
  avg_arr.append(file.shape[0])

print(np.max(avg_arr))
print(np.min(avg_arr))
print(np.mean(avg_arr))
print(np.std(avg_arr))

2197
328
798.515625
350.84220064846727


##Visualizing the value of landmarks coordinates for a random video and random landmark

In [ ]:
idx = np.random.randint(0,64)
print('File_index_no:',idx)


file_path = os.path.join(extracted_feature_path,filtered_list_npy[idx])

# print(file_path)

file = np.load(file_path)
print(file.shape[0])

lnd_idx = np.random.randint(0,33)
print('landmark_index:',lnd_idx)


for i in range(0,file.shape[0],50):
  print(f"{i}:x-coordinate",file[i,lnd_idx,0])
print('\n')
for i in range(0,file.shape[0],50):
  print(f"{i}:y-coordinate",file[i,lnd_idx,1])
print('\n')

for i in range(0,file.shape[0],50):
  print(f"{i}:z-coordinate",file[i,lnd_idx,2])
print('\n')



File_index_no: 36
1634
landmark_index: 32
0:x-coordinate 0.450382262468338
50:x-coordinate 0.44737809896469116
100:x-coordinate 0.4482800364494324
150:x-coordinate 0.44784078001976013
200:x-coordinate 0.4491288363933563
250:x-coordinate 0.44849860668182373
300:x-coordinate 0.4507908225059509
350:x-coordinate 0.44729363918304443
400:x-coordinate 0.45188573002815247
450:x-coordinate 0.4487418532371521
500:x-coordinate 0.4516087472438812
550:x-coordinate 0.4481900930404663
600:x-coordinate 0.44980141520500183
650:x-coordinate 0.44753745198249817
700:x-coordinate 0.4508907198905945
750:x-coordinate 0.44840580224990845
800:x-coordinate 0.4518798291683197
850:x-coordinate 0.4474010169506073
900:x-coordinate 0.44829851388931274
950:x-coordinate 0.4477080702781677
1000:x-coordinate 0.44825229048728943
1050:x-coordinate 0.4476023316383362
1100:x-coordinate 0.44773563742637634
1150:x-coordinate 0.44721654057502747
1200:x-coordinate 0.4482359290122986
1250:x-coordinate 0.45016369223594666
1300:x-

##Splitting a single .npy file into file of 30 frames

In [ ]:
len(values_list)|

64

In [ ]:
count = 0
X = np.empty((0, 30, 33, 3), dtype=np.float32)
Y = []

for idx, file_name in enumerate(filtered_list_npy):
    file_path = os.path.join(extracted_feature_path, file_name)
    file = np.load(file_path)
    label = values_list[idx]
    print("label:", label)

    N = file.shape[0]
    print(f'no.of frames in file {file_name} is:', N)
    print(30 * (N // 30))

    num_frames_per_dp = 30
    num_groups = (N ) // num_frames_per_dp

    augmented_frames = np.zeros((num_groups, num_frames_per_dp))
    print(augmented_frames.shape)

    for i in range(0, num_groups):
        for j in range(0, 30):
            augmented_frames[i, j] = i + j * num_groups

    print('Shape of augmented frames', augmented_frames.shape)
    print(augmented_frames[0])

    for grp in augmented_frames:
        x = []
        for frame in grp:
            x.append(file[int(frame)])

        X = np.append(X, [x], axis=0)
        Y.append(label)


# Now X should have the shape (300, 30, 33, 3)
print('Shape of X:', X.shape)

label: 1
no.of frames in file alli1.mov_landmarks.npy is: 949
930
(31, 30)
Shape of augmented frames (31, 30)
[  0.  31.  62.  93. 124. 155. 186. 217. 248. 279. 310. 341. 372. 403.
 434. 465. 496. 527. 558. 589. 620. 651. 682. 713. 744. 775. 806. 837.
 868. 899.]
label: 1
no.of frames in file alli2.mov_landmarks.npy is: 941
930
(31, 30)
Shape of augmented frames (31, 30)
[  0.  31.  62.  93. 124. 155. 186. 217. 248. 279. 310. 341. 372. 403.
 434. 465. 496. 527. 558. 589. 620. 651. 682. 713. 744. 775. 806. 837.
 868. 899.]
label: 1
no.of frames in file anan1.mov_landmarks.npy is: 778
750
(25, 30)
Shape of augmented frames (25, 30)
[  0.  25.  50.  75. 100. 125. 150. 175. 200. 225. 250. 275. 300. 325.
 350. 375. 400. 425. 450. 475. 500. 525. 550. 575. 600. 625. 650. 675.
 700. 725.]
label: 1
no.of frames in file anan2.mov_landmarks.npy is: 948
930
(31, 30)
Shape of augmented frames (31, 30)
[  0.  31.  62.  93. 124. 155. 186. 217. 248. 279. 310. 341. 372. 403.
 434. 465. 496. 527. 558. 5

In [ ]:
os.chdir('/content/drive/MyDrive/Anit/Gym_Posture_Data/train_test')

In [ ]:
!pwd

#Saving the files into X and Y
np.save('X.npy', X)
np.save('Y.npy', Y)


/content/drive/MyDrive/Anit/Gym_Posture_Data/train_test


In [ ]:
split_index = int(len(Y) * 0.8)

X_train=np.empty((0, 30, 33, 3), dtype=np.float32)
X_test= np.empty((0, 30, 33, 3), dtype=np.float32)

X_train = X[:split_index]
Y_train = Y[:split_index]

X_test = X[split_index:]
Y_test = Y[split_index:]

print(X_train.shape)
print(len(Y_train))
print(X_test.shape)
print(len(Y_test))

np.save('X_train.npy',X_train)
np.save('Y_train.npy',Y_train)
np.save('X_test.npy',X_test)
np.save('Y_test.npy',Y_test)

(1339, 30, 33, 3)
1339
(335, 30, 33, 3)
335


##Training LSTM

In [ ]:
np.load(filtered_list_npy[0]).shape

X = []
y=values_list

for file in filtered_list_npy:
    X.append(np.load(file))


In [ ]:
#Model

# Define your LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Hyperparameters
input_size = 33
hidden_size = 64
num_layers = 2
num_classes = 2
num_epochs = 10
learning_rate = 0.001


In [ ]:
# Convert NumPy arrays to lists of tensors
X_tensors = [torch.tensor(x, dtype=torch.float32) for x in X]
y_tensors = [torch.tensor(label, dtype=torch.long) for label in y]

# Get the sequence lengths
sequence_lengths = [x_tensor.shape[0] for x_tensor in X_tensors]

# Sort tensors by sequence length (descending order)
sorted_indices = sorted(range(len(sequence_lengths)), key=lambda i: -sequence_lengths[i])
X_tensors = [X_tensors[i] for i in sorted_indices]
y_tensors = [y_tensors[i] for i in sorted_indices]
sequence_lengths = [sequence_lengths[i] for i in sorted_indices]

# Create packed sequences
X_packed = pack_padded_sequence(torch.stack(X_tensors), sequence_lengths, batch_first=True)
y_packed = torch.stack(y_tensors)

# Define a new dataset with packed sequences
dataset = TensorDataset(X_packed, y_packed)

RuntimeError: ignored

In [ ]:

# Split the data into training, validation, and test sets
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with model checkpointing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

best_val_loss = float('inf')
model_checkpoint_path = 'best_model.pth'

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_train_loss = total_loss / len(train_loader)

    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

    average_val_loss = total_val_loss / len(val_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Training Loss: {average_train_loss:.4f}, '
          f'Validation Loss: {average_val_loss:.4f}')

    # Save the model if the validation loss is improved
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        torch.save(model.state_dict(), model_checkpoint_path)
        print("Model saved")

# Load the best model for testing
best_model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
best_model.load_state_dict(torch.load(model_checkpoint_path))
best_model.to(device)

# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Evaluate the model on the test set
best_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = best_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate the F1-score
f1 = f1_score(true_labels, predicted_labels)

print(f'F1-score on the test set: {f1:.4f}')

AttributeError: ignored

In [ ]:
##Vishal dekh paa rha hai kya

## Ha bhai